In [1]:
from helpers.data import get_dataloaders
from helpers.train import TrainingManager
from helpers.loss_accuracy import accuracy
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import torch

In [2]:
device = torch.device('cuda:6')

In [3]:
from efficientnet_pytorch import EfficientNet
ef_model = 7
ef_image_input = [
    224,
#     240,
#     260,
#     300,
#     380,
#     456,
#     528,
#     600,
#     672,
#     800
]
model = nn.Sequential(nn.UpsamplingBilinear2d(size=(ef_image_input[ef_model],ef_image_input[ef_model])),
                      EfficientNet.from_pretrained(f'efficientnet-b{ef_model}', 10)).to(device)

Loaded pretrained weights for efficientnet-b7


In [16]:
parameters_to_train = [name for name,i in model.named_parameters()]
print(len(parameters_to_train))
first_last=250
parameters_to_train = parameters_to_train[:first_last] + parameters_to_train[-4:]

711


In [17]:
parameters = [i for name,i in model.named_parameters() if name in parameters_to_train]

In [18]:
len(parameters) == len(parameters_to_train)

True

In [19]:
trainloader, testloader = get_dataloaders('cifar10', 2)

Files already downloaded and verified


In [24]:
optimizer = optim.SGD(parameters, lr=1e-5, momentum=0.9, nesterov=True)
def lrfn(from_, to_, cycle_):
    def lr_(step):
        step = step % (cycle_ * 2)
        if step >= cycle_:
            step = cycle_ - (step - cycle_)  
        return 10 ** (from_ + (step / cycle_) * (to_ - from_))
    return lr_
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lrfn(-3,-1,200))

In [25]:
trial_name = f"efficient_net_b{ef_model}_pytorch_cifar10_transfer_learning"

In [26]:
tm = TrainingManager(trial_name, load=False, is_trial=False)

In [27]:
tm.train(model, optimizer,
         trainloader, testloader,
         CrossEntropyLoss(), CrossEntropyLoss(),
         accuracy, accuracy, lr_scheduler=lr_scheduler, device=device, no_iterations=51000)

  0%|          | 0/51000 [00:00<?, ?it/s]

Start training trial: efficient_net_b7_pytorch_cifar10_transfer_learning is_trial


{tr_loss: 2.30228, tr_acc: 0.10924, te_loss: 2.31412, te_acc: 0.08403}:   0%|          | 133/51000 [00:44<4:41:31,  3.01it/s]


KeyboardInterrupt: 